<h1 style="color:#8BB065;font-size:50px;"><strong>My first <strong style="color:#974949"> Convolutional Neural Network</strong></strong></h1>


<img src="https://i.ibb.co/VMSgnJ5/header.png" alt="header" border="0">

Learning CNN from this notebook: [Link](https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1)

In [ ]:
#load in packages
import os
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import gc
from tqdm import tqdm

#images
import cv2

#modeling
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

#visualizations
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#source path (where the Pawpularity contest data resides)
path = '../input/petfinder-pawpularity-score/'

#Get the metadata (the .csv data) and put it into DataFrames
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')

#Get the image data (the .jpg data) and put it into lists of filenames
train_jpg = glob(path + "train/*.jpg")
test_jpg = glob(path + "test/*.jpg")

In [ ]:
train_df.shape, len(train_jpg)

In [ ]:
train_df.head(5)

<h1 style="color:#189AB4;font-size:20px;"><strong>EDA<strong style="color:black"></strong></strong></h1>

In [ ]:
sns.histplot(train_df['Pawpularity']);

plt.axvline(train_df['Pawpularity'].mean(),color='red')
plt.axvline(train_df['Pawpularity'].median(),color='green')

In [ ]:
train_df[['Pawpularity']].describe().T

In [ ]:
train_df.columns

In [ ]:
cols = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

for col in cols:
    fig, ax = plt.subplots(1,2)
    sns.violinplot(data = train_df,y ='Pawpularity', x=col, ax=ax[0])
    sns.histplot(data = train_df,x ='Pawpularity',hue=col,kde=True,fill=True, ax=ax[1])
    
    plt.suptitle(str(col), fontsize=18, fontweight='bold')
    fig.show()

In [ ]:
img_path = '../input/petfinder-pawpularity-score/train/'
ext = '.jpg'

nums = len(cols)

# col = 'Eyes'
for count, col in enumerate(cols):
    sample = train_df.loc[train_df[col] == 1,'Id'].head(100).values[np.random.randint(10)]
    
    pawpularity = train_df.loc[train_df['Id'] == sample, 'Pawpularity'].head(1).values[-1]
    
    image_loc = img_path + sample + ext

    image_array = plt.imread(image_loc)
    plt.imshow(image_array)
    
    plt.title(f'Image of pet with {col}\nPawpularity Score: {pawpularity}') 
    plt.axis('off') #turns off the gridlines
    plt.show()

    del sample, image_loc, image_array
    gc.collect()

In [ ]:
#Modify the Id such that each Id is the full image path. In the form
def train_id_to_path(x):
    return '../input/petfinder-pawpularity-score/train/' + x + ".jpg"
def test_id_to_path(x):
    return '../input/petfinder-pawpularity-score/test/' + x + ".jpg"


In [ ]:
#Check to see
tf.config.get_visible_devices()

In [ ]:
#Read in the data and drop unnecessary columns
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train = train.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test = test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

In [ ]:
#Add the .jpg extensions to the image file name ids
train["img_path"] = train["Id"].apply(train_id_to_path)
test["img_path"] = test["Id"].apply(test_id_to_path)

In [ ]:
#binning columns to test models
train['two_bin_pawp'] = pd.qcut(train['Pawpularity'], q=2, labels=False)
train = train.astype({"two_bin_pawp": str})

train['four_bin_pawp'] = pd.qcut(train['Pawpularity'], q=4, labels=False)
train = train.astype({"four_bin_pawp": str})

train['ten_bin_pawp'] = pd.qcut(train['Pawpularity'], q=10, labels=False)
train = train.astype({"ten_bin_pawp": str})

In [ ]:
#Set the size image you want to use
image_height = 128
image_width = 128

#define a function that accepts an image url and outputs an eager tensor
def path_to_eagertensor(image_path):
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    #image = tf.image.resize_with_pad(image, image_height, image_width) #optional with padding to retain original dimensions
    image = tf.image.resize(image, (image_height, image_width))
    return image

In [ ]:
#get all the images in the training folder and put their tensors in a list
X = []
for img in tqdm(train['img_path']):
    new_img_tensor = path_to_eagertensor(img)
    X.append(new_img_tensor)
    
print(type(X),len(X))
X = np.array(X)
print(type(X),X.shape)

In [ ]:
#get all the images in the test folder and put their tensors in a list
X_submission = []
for img in tqdm(test['img_path']):
    new_img_tensor = path_to_eagertensor(img)
    X_submission.append(new_img_tensor)
    
print(type(X_submission),len(X_submission))
X_submission = np.array(X_submission)
print(type(X_submission),X_submission.shape)

In [ ]:
#grab the target variable. In our case, Pawpularity
y = train['Pawpularity']
print(type(y))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7)

In [ ]:
#Show the shape of each of the new arrays
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
def get_model(SEED=42):
    tf.keras.backend.clear_session()
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    
    input_ = keras.Input(shape=(X.shape[1],X.shape[2],3))

    x = keras.layers.Conv2D(filters = 16, kernel_size=[7,7], strides=[2,2], padding='valid', kernel_initializer='he_normal',
                           kernel_regularizer=l2(0.0005), activation = 'relu')(input_)

    x = keras.layers.Conv2D(filters = 32, kernel_size=[3,3], padding='same', kernel_initializer='he_normal',
                           activation = 'relu')(x)

    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Conv2D(filters = 32, kernel_size=[3,3],strides=[2,2], padding='same', kernel_initializer='he_normal',
                           activation = 'relu', kernel_regularizer=l2(0.0005))(x)

    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Dropout(0.25)(x)

    x = keras.layers.Conv2D(filters = 64, kernel_size=[3,3], padding='same', kernel_initializer='he_normal', activation='relu'
                            ,kernel_regularizer=l2(0.0002))(x)

    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same',
                               kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)

    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dropout(0.25)(x)

    #####
    x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)

    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)

    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Flatten()(x)

    x = tf.keras.layers.Dense(512, activation = "relu")(x)

    x = tf.keras.layers.Dropout(0.5)(x)

    output = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs = input_, outputs = output)
    
    return model

In [ ]:
model = get_model()
model.summary()

In [ ]:
#compile the model
model.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse")])

In [ ]:
data_augmentation = ImageDataGenerator(rotation_range=15, zoom_range=0.15, width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")

In [ ]:
kall = keras.callbacks.EarlyStopping(monitor='val_rmse',patience=10,restore_best_weights=True)

history = model.fit(
    data_augmentation.flow(x_train,y_train,batch_size=1024),
    validation_data = (x_test,y_test),
    steps_per_epoch = len(x_train) // 1024,
    epochs = 600, callbacks=[kall]
)

In [ ]:
plt.figure()
plt.plot(history.history["rmse"], label="train_rmse")
plt.plot(history.history["val_rmse"], label="val_rmse")
plt.title("RMSE train/validation by Epoch")
plt.xlabel("Epoch #")
plt.ylabel("RMSE")
plt.legend(loc="upper right");

In [ ]:
#predict on the submission data
cnn_pred = model.predict(X_submission)
print(X_submission.shape, type(X_submission))
print(cnn_pred.shape, type(cnn_pred))

In [ ]:
#put the submission predictions alongside their associated Ids
cnn = pd.DataFrame()
cnn['Id'] = test['Id']
cnn['Pawpularity'] = cnn_pred
cnn.to_csv('submission.csv',index=False)

In [ ]:
testing_example_image = plt.imread('../input/petfinder-pawpularity-score/test/4128bae22183829d2b5fea10effdb0c3.jpg') 
print(testing_example_image.shape)
#then plt.imshow() can display it for you
plt.imshow(testing_example_image)
plt.title('First Testing Image \n Predicted Pawpularity = {}'.format(cnn['Pawpularity'].iloc[0])) 
plt.axis('off') #turns off the gridlines
plt.show()